### **핵심 주제**
---
1. **다수결 분류기**
2. **배깅/페이스팅**
  * 랜덤 패치와 랜덤 서브스페이스
3. **랜덤 포레스트**
4. **부스팅**
  * 에이다부스트
  * 그레이디언트 부스팅

# 1. **다수결 분류기**
---
* **다수결 분류기**(voting classifier)는  동일한 훈련 세트에 대해 **여러 종류의 분류기를 학습**시킨 후 각 분류기의 예측을 모아 **다수결** 방식으로 예측값을 결정
  * 이와 같이 작동하는 분류기를 **직접 투표(hard voting) 분류기**라 부름

* 사용되는 분류기가 모두 **클래스에 포함될 확률을 계산**하는 **`predict_proba()`** 메서드를 지원한다면 사용된 분류기의 확률 예측을 평균 계산해 **가장 높은 확률의 클래스**를 선택 가능 
  * 이와 같이 작동하는 분류기를 **간접 투표(soft voting) 분류기**라 부름
  * 확률이 높은 투표에 비중을 더 두기 때문에 직접 투표 방식보다 좀 더 좋은 성능을 발휘

* 다수결 분류기의 기본적인 작동 과정
![다수결 분류기 작동 과정](https://github.com/liganega/handson-ml2/raw/7dfe4edf750309a691d5977308ed845cb5f526c4/notebooks/images/ensembles/majority_voting.png)

* 다수결 분류기의 특징
  * 다수결 분류기에 포함된 분류기들 사이의 **독립성이 전재**된다면, 하나의 성능이 좋은 분류기를 사용하는 것보다 **정확한 예측**이 가능
    * 독립성이 보장되지 못한다면 다수결 분류기의 성능이 더 낮아질 수 있음

* 사이킷런의 다수결 분류기
  * **`VotingClassifier`**이 다수결 분류기 모델을 제공
  * `voting='hard'`: 직접 투표 방식
    * 간접 투표 방식은 `'hard'` 대신 `'soft'`로 지정

# 2. **배깅/페이스팅**
---
* 훈련 세트의 **부분집합**을 임의로 구성한 후 **동일한 분류기를 생성된 부분 집합을 이용하여 학습**시키는 방식으로 **앙상블** 학습 진행 가능
* 훈련 세트의 부분집합을 임의로 선택할 때 중복 허용 여부에 따라 앙상블 학습 방식이 달라짐
  * **배깅**: 중복 허용 샘플링
  * **페이스팅**: 중복 미허용 샘플링

* 배깅 방식 묘사
![배깅 방식 묘사](https://github.com/liganega/handson-ml2/raw/7dfe4edf750309a691d5977308ed845cb5f526c4/notebooks/images/ensembles/Bagging.png)

##2.1 **앙상블 학습 예측 방식**
---
- 최종적으로 **종합 함수**(aggregation function)가 각 개별기의 결과를 종합해서 최종 예측값을 지정
  * **분류** 모델의 경우: **직접 투표** 방식 사용. 즉, 수집된 예측값들 중에서 **최빈값**(mode) 선택
  * **회귀** 모델의 경우: 수집된 예측값들의 평균값 선택

####**편향과 분산**
* 앙상블 학습을 이용한 예측기의 경우 개별 예측기의 경우에 비해 **편향은 비슷하지만 분산은 줄어듦**
  * 과대적합의 위험성이 줄어듦

* 개별 학습기의 경우, **배깅/페이스팅 방식**으로 학습하면 전체 훈련 세트를 대상으로 학습한 경우에 비해 **편향이 커짐**
  * 과소적합 위험성이 커짐


## 2.2 **사이킷런의 배깅/페이스팅**
---
* 사이킷런에서 제공하는 배깅/페이스팅 모델
  * 분류 모델: `BaggingClassifier`
  * 회귀 모델: `BaggingRegressor`

#### **배깅 대 페이스팅**
* **중복선택을 허용**하는 배깅 방식이 사용되는 샘플 부분집합의 다양성을 좀 더 키우면, 다음 효과를 가져옴
  * 일반적으로 배깅 방식이 **편향은 키우고, 분산은 줄임**
    * 표본 샘플링의 무작위성은 커지면서 예측기들 사이의 상관관계 정도가 약화되기 때문
  * 전반적으로 **배깅 방식**이 좀 더 나은 모델 생성
  * **교차검증**을 통해 항상 확인 필요

#### **oob 평가**
* **oob(out-of-bag)** 샘플: 선택되지 않은 훈련 샘플
* oob 샘플은 예측기마다 다르며, 예측기가 학습 중에 oob 샘플을 사용하지 않으므로 이 샘플들을 **검증 세트로 사용**할 수 있음
* 앙상블 모델 자체는 예측기별 oob 평가의 평균값을 이용하여 평가할 수 있음
* `BaggingClassifier` 의 **`oob_score=True`** 로 설정하면 oob 평가를 자동으로 실행

## 2.3 **랜덤 패치/랜덤 서브스페이스**
---
* `BaggingClassifier`는 특성을 대상으로 하는 두 개의 하이퍼파라미터를 제공
  * **`max_features`**
    * 학습에 사용할 특성 수를 지정
    * 특성 선택은 무작위
      * 정수인 경우: 지정된 수만큼 특성 선택
      * 부동소수점($\in [0, 1]$)인 경우: 지정된 비율만큼 특성 선택
    * max_samples와 유사 기능 수행
  * **`bootstrap_features`**:
    * 학습에 사용할 특성을 선택할 때 중복 허용 여부 지정
    * 기본값은 False
    * botostrap과 유사 기능 수행


* 이미지 등과 같이 매우 높은 차원의 데이터셋을 다룰 때 유용하게 사용
* **특성 샘플링**은 더 다양한 예측기를 만들며, **편향이 커지지만 분산은 낮아짐**
  * 적은 수의 특성을 사용하면 아무래도 예측이 부정확해질 수밖에 없는 반면에, 데이터 샘플에 대해 보다 덜 민감하게 반응하게 되기 때문

#### **랜덤 패치 기법**
* 훈련 샘플과 훈련 특성 모두를 대상으로 **중복을 허용**하며 **임의의 샘플 수와 임의의 특성 수만큼을 샘플링**해서 학습하는 기법
* 아래 두 조건이 참이어야 함
  * `bootstrap=True` 또는 `max_samples < 1.0`
  * `bootstrap_features=True` 또는 `max_features < 1.0`


#### **랜덤 서브스페이스 기법**
* 전체 훈련 세트를 학습 대상으로 삼지만 **훈련 특성은 임의의 특성 수만큼 샘플링**해서 학습하는 기법
* 아래 두 조건이 참이어야 함
  * `bootstrap=False` 그리고 `max_samples=1.0`
  * `bootstrap_features=True` 또는 `max_features < 1.0`

# 3. **랜덤 포레스트**
---
* **랜덤 포레스트**(random forest): 배깅/페이스팅 방법을 적용한 의사결정나무의 앙상블을 최적화한 모델
* 사이킷런의 **`RandomFrestClassifier`**를 이용해 랜덤 포레스트를 구현
  * 회귀용 예측기는 ` RandomForestRegressor` 가 지원

#### **엑스트라 트리**
* 랜덤 포레스트는 의사결정나무의 마디를 결정할 때 사용할 특성을 **무작위로 선택된 특성과 임곗값 전체를 대상으로** 해서 그 중에 **최적의 조합**을 선택
  * 의사결정나무의 아래 옵션을 사용하는 것과 동일
    * **`splitter="best"`**: 의사결정나무의 마디 속성을 정할 때 사용 대상 특성 전체를 대상으로해서 최적의 특성과 임곗값 선택
    * **`max_features="auto"`**: 학습에 사용되는 특성의 수를 전체 특성 수의 제곱근 값으로 제한

* **엑스트라 트리(extra-trees)**
  * 최적의 임곗값을 찾는 대신 선택된 특성 중에서 일부만을 무작위로 대상으로 삼아 **임계값을 무작위로 선택해서 샘플을 분할**한 다음 그 중에 **최선의 분할**을 선택하는 방식
  * 아래 옵션과 함께 사용
    * **`splitter="random"`**: 의사결정나무의 마디 지정에 사용될 특성과 임곗값을 무작위로 지정해서 분할한 다음 그 중에 최선의 분할 선택
    * **`max_features="auto"`**: 학습에 사용되는 특성의 수를 전체 특성 수의 제곱근 값으로 제한
  
  * 일반적인 랜덤 포레스트보다 속도가 훨씬 빠름
  * 이 방식을 사용하면 **편향은 늘고, 분산은 줄어듦**


#### **특성 중요도**
* 랜덤 포레스트는 학습에 사용된 특성의 상대적 중요도도 함께 측정
* **특성 중요도**: 해당 특성을 사용한 마디가 **평균적으로 불순도를 얼마나 감소시키는지**를 측정
  * 즉, 불순도를 많이 줄이면 그만큼 중요도가 커짐
* 사이킷런의 **`RandomForestClassifier`**: 훈련이 끝난 뒤 특성별 중요도의 **전체 합이 1**이 되도록 하는 방식
  * 특성별 상대적 중요도를 측정한 후 `feature_importances_` 속성에 저장

# 4. **부스팅**
---
* **부스팅(boosting)**: 성능이 약한 학습기의 여러 개를 선형으로 연결하여 강한 성능의 학습기를 만드는 앙상블 기법
  * **순차적으로** 이전 학습기의 결과를 바탕으로 **성능을 조금씩 높혀가는** 방식
  * 배깅/페이스팅에 비해 확장성이 떨어짐

* 성능이 약한 예측기의 단점을 보완하여 좋은 성능의 예측기를 훈련해 나가는 것이 부스팅의 기본 아이디어

## 4.1 **에이다부스트(AdaBoost)**
---
* 이전 모델이 제대로 **학습하지 못한 샘플들**에 대한 **가중치를 조절**하여 새로운 모델을 만들어 가는 방식
* 새로운 예측기는 학습하기 어려운 샘플에 조금씩 더 잘 적응하는 모델이 연속적으로 만들어져 감
![에이다부스트](https://github.com/liganega/handson-ml2/raw/7dfe4edf750309a691d5977308ed845cb5f526c4/notebooks/images/ensembles/Adaboost.png)

* **사이키런의 에이다부스트**
  * 사이킷런은 `AdaBoostClassifier` 분류 모델과 `AdaBoostRegressor` 회귀 모델을 지원

## 4.2 **그레이디언트 부스팅**
---
* 이전 학습기에 의한 오차를 보정하도록 새로운 예측기를 순차적으로 추가하는 아이디어는 에이다부스트와 동일
* 샘플의 가중치를 수정하는 대신 이전 예측기가 만든 **나머지 오차(residual error)에 대해 새로운 예측기를 학습**시킴



#### **사이킷런 그레이디언트 부스팅 모델**
* 사이킷런의 **`GradientBoostingRegressor`** 클래스가 제공
  * `RandomForestRegressor`와 비슷한 하이퍼파라미터를 제공

#### **`learning_rate`(학습률)**
  * `learnign_rate`는 기존에 설명한 학습률과 다른 의미의 학습률
    * 기존 학습률: 하나의 모델이 학습되는 과정에서 샘플 특성의 가중치를 업데이트할 때 사용되는 손실함수의 그레이디언트 비율 지정
    * 이전 의사결정나무에서 학습된 값을 전달할 때 사용되는 비율
      * 1.0이면 그대로 전달
      * 1.0보다 작으면 해당 비율 만큼 조금만 전달
  * 그레이디언트 부스팅 알고리즘은 모델 내부의 가중치가 아닌 **모델 자체를 새로운 모델로 업데이트 하면서 최적의 모델을 찾아가는 과정**
  * 이때, 나머지 오차는 특정 손실함수의 그레이디언트와 연관되어 있음
    * **나머지 오차를 얼마의 비율로 사용**하여 새로운 모델로 업데이트할까를 정하는 문제는 기존의 학습률과 비슷


#### 축소 규제
* **학습률**(`learning_rate`)을 이용하여 학습 과정을 조절하는 방식
* 학습률을 낮추면 많은 의사결정나무가 필요하지만 일반적으로 **예측 성능이 좋아짐**